In [2]:
import csv
import pandas as pd
import networkx as nx

In [3]:
data_file = '/global/D1/projects/umod/dipp/Misinformation_Dataset/Result/location_with_value/Location_with_only_country_code_cat_4_only_3.csv'
population_data_file = '/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Country_Data_Extraction/Result/Country_Data_cat_4_only_3.csv'

In [4]:
## Includes self Loops

country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            
            souce_loc = row['i'].replace("'","")
            destination_loc = row['j'].replace("'","")
            #country_data[count] = [row['i'], row['j'], float(row['count'])]
            #if(float(row['count']) > 999):
            country_data[count] = [souce_loc, destination_loc, float(row['count'])]
            count+=1

In [5]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['Source','Destination','Number of Tweets'])

In [6]:
df

,Source,Destination,Number of Tweets
0,CH,CH,111.0
1,GB,GB,5043.0
2,US,US,26779.0
3,PL,PL,11.0
4,IE,IE,63.0
...,...,...,...
656,MX,IE,1.0
657,SI,US,1.0
658,ES,PT,1.0
659,TH,GB,1.0


## Creating Graph using Networkx

In [7]:
G = nx.from_pandas_edgelist(df, source='Source', target='Destination', edge_attr='Number of Tweets', create_using=nx.DiGraph())

In [8]:
G.degree(weight='Number of Tweets')['US']
G.in_degree(weight='Number of Tweets')['US']
G.out_degree(weight='Number of Tweets')['US']

30216.0

In [9]:
country = []
deg = G.degree(weight='Number of Tweets')
deg_in = G.in_degree(weight='Number of Tweets')
deg_out = G.out_degree(weight='Number of Tweets')

In [10]:
df['All Degree of Source'] = ''
df['All Degree of Destination'] = ''
df['Vi - Out Degree of Source'] = ''
df['Vj - In Degree of Destination'] = ''
# df['Percentage of Incoming'] = ''
# df['Total No. of Tweets/Population'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,111.0,,,,
1,GB,GB,5043.0,,,,
2,US,US,26779.0,,,,
3,PL,PL,11.0,,,,
4,IE,IE,63.0,,,,
...,...,...,...,...,...,...,...
656,MX,IE,1.0,,,,
657,SI,US,1.0,,,,
658,ES,PT,1.0,,,,
659,TH,GB,1.0,,,,


In [11]:
for index_df, row in df.iterrows():
    i = row['Source']
    j = row['Destination']

    for country_temp, deg_all in deg:
        if(country_temp == i):
            df.at[index_df, 'All Degree of Source'] = deg_all
        if(country_temp == j):
            df.at[index_df, 'All Degree of Destination'] = deg_all
    
    # Vj is in degree of destination

    for country_temp_in, deg_j in deg_in:
        if(country_temp_in == j):
            df.at[index_df, 'Vj - In Degree of Destination'] = deg_j
        # else:
        #     df.at[index_df, 'Vj - In Degree of Destination'] = 0
        

    # Vi is out degree of source

    for country_temp_out, deg_i in deg_out:
        if(country_temp_out == i):
            df.at[index_df, 'Vi - Out Degree of Source'] = deg_i
        # else:
        #     df.at[index_df, 'Vi - Out Degree of Source'] = 0    


In [12]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination
0,CH,CH,111.0,387.0,387.0,208.0,179.0
1,GB,GB,5043.0,13998.0,13998.0,7134.0,6864.0
2,US,US,26779.0,60559.0,60559.0,30216.0,30343.0
3,PL,PL,11.0,72.0,72.0,29.0,43.0
4,IE,IE,63.0,265.0,265.0,143.0,122.0
...,...,...,...,...,...,...,...
656,MX,IE,1.0,399.0,265.0,217.0,122.0
657,SI,US,1.0,7.0,60559.0,4.0,30343.0
658,ES,PT,1.0,109.0,122.0,48.0,95.0
659,TH,GB,1.0,12.0,13998.0,3.0,6864.0


In [13]:
df['Percentage of Incoming at Destination (Total Tweet / Vj)'] = ''
df['Percentage of Outgoing From Source (Total Tweet / Vi)'] = ''
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,111.0,387.0,387.0,208.0,179.0,,
1,GB,GB,5043.0,13998.0,13998.0,7134.0,6864.0,,
2,US,US,26779.0,60559.0,60559.0,30216.0,30343.0,,
3,PL,PL,11.0,72.0,72.0,29.0,43.0,,
4,IE,IE,63.0,265.0,265.0,143.0,122.0,,
...,...,...,...,...,...,...,...,...,...
656,MX,IE,1.0,399.0,265.0,217.0,122.0,,
657,SI,US,1.0,7.0,60559.0,4.0,30343.0,,
658,ES,PT,1.0,109.0,122.0,48.0,95.0,,
659,TH,GB,1.0,12.0,13998.0,3.0,6864.0,,


In [14]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']

    per_in = float(No_of_Tweets / vj) * 100
    per_out = float(No_of_Tweets / vi) * 100
    df.at[index_df, 'Percentage of Incoming at Destination (Total Tweet / Vj)'] = per_in
    df.at[index_df, 'Percentage of Outgoing From Source (Total Tweet / Vi)'] = per_out

    


In [15]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi)
0,CH,CH,111.0,387.0,387.0,208.0,179.0,62.011173,53.365385
1,GB,GB,5043.0,13998.0,13998.0,7134.0,6864.0,73.47028,70.689655
2,US,US,26779.0,60559.0,60559.0,30216.0,30343.0,88.254293,88.625232
3,PL,PL,11.0,72.0,72.0,29.0,43.0,25.581395,37.931034
4,IE,IE,63.0,265.0,265.0,143.0,122.0,51.639344,44.055944
...,...,...,...,...,...,...,...,...,...
656,MX,IE,1.0,399.0,265.0,217.0,122.0,0.819672,0.460829
657,SI,US,1.0,7.0,60559.0,4.0,30343.0,0.003296,25.0
658,ES,PT,1.0,109.0,122.0,48.0,95.0,1.052632,2.083333
659,TH,GB,1.0,12.0,13998.0,3.0,6864.0,0.014569,33.333333


## Extensions

### Tweets per population and 2 versions of Cij 

In [16]:
population_data = {}

with open(population_data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        # print(row['Country Code'])
        # print(row['Population'])
        if(row['Population']!=''):
            population_data[row['Country Code']] = int(row['Population'])


In [17]:
population_data['AI'] = 15753
population_data['AQ'] = 3000
population_data['AX'] = 30129
population_data['BQ'] = 27726
population_data['CX'] = 1692
population_data['EH'] = 612000
population_data['FK'] = 3662
population_data['GF'] = 301099
population_data['GG'] = 63950
population_data['GS'] = 1
population_data['IO'] = 3000
population_data['JE'] = 103267
population_data['PN'] = 47
population_data['RE'] = 873102
population_data['SH'] = 4439
population_data['TF'] = 300
population_data['TW'] = 23894394
population_data['UM'] = 300
population_data['VA'] = 453
population_data['YT'] = 310022

In [18]:
df['Tweets Per Population'] = ''
df['Cij (All Degrees)'] = ''
df['Cij'] = ''

In [19]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,111.0,387.0,387.0,208.0,179.0,62.011173,53.365385,,,
1,GB,GB,5043.0,13998.0,13998.0,7134.0,6864.0,73.47028,70.689655,,,
2,US,US,26779.0,60559.0,60559.0,30216.0,30343.0,88.254293,88.625232,,,
3,PL,PL,11.0,72.0,72.0,29.0,43.0,25.581395,37.931034,,,
4,IE,IE,63.0,265.0,265.0,143.0,122.0,51.639344,44.055944,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
656,MX,IE,1.0,399.0,265.0,217.0,122.0,0.819672,0.460829,,,
657,SI,US,1.0,7.0,60559.0,4.0,30343.0,0.003296,25.0,,,
658,ES,PT,1.0,109.0,122.0,48.0,95.0,1.052632,2.083333,,,
659,TH,GB,1.0,12.0,13998.0,3.0,6864.0,0.014569,33.333333,,,


In [20]:
import pypopulation

In [21]:
temp = 0
None_Type_Countries = []
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']


    # source_poplation = population_data.get(source)
    # dest_population = population_data.get(destination)


    source_poplation = pypopulation.get_population(source)
    dest_population = pypopulation.get_population(destination)

    if(source_poplation == None):
        source_poplation = population_data.get(source)
            

    if(dest_population == None):
        dest_population = population_data.get(destination)
            

    percentage = No_of_Tweets / (source_poplation + dest_population)

    df.at[index_df, 'Tweets Per Population'] = percentage

    

In [22]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,111.0,387.0,387.0,208.0,179.0,62.011173,53.365385,0.000006,,
1,GB,GB,5043.0,13998.0,13998.0,7134.0,6864.0,73.47028,70.689655,0.000038,,
2,US,US,26779.0,60559.0,60559.0,30216.0,30343.0,88.254293,88.625232,0.000041,,
3,PL,PL,11.0,72.0,72.0,29.0,43.0,25.581395,37.931034,0.0,,
4,IE,IE,63.0,265.0,265.0,143.0,122.0,51.639344,44.055944,0.000006,,
...,...,...,...,...,...,...,...,...,...,...,...,...
656,MX,IE,1.0,399.0,265.0,217.0,122.0,0.819672,0.460829,0.0,,
657,SI,US,1.0,7.0,60559.0,4.0,30343.0,0.003296,25.0,0.0,,
658,ES,PT,1.0,109.0,122.0,48.0,95.0,1.052632,2.083333,0.0,,
659,TH,GB,1.0,12.0,13998.0,3.0,6864.0,0.014569,33.333333,0.0,,


In [23]:
for index_df, row in df.iterrows():
    source = row['Source']
    destination = row['Destination']
    No_of_Tweets = row['Number of Tweets']
    all_degree_source = row['All Degree of Source']
    all_degree_dest = row['All Degree of Destination']
    vi = row['Vi - Out Degree of Source']
    vj = row['Vj - In Degree of Destination']


    cij_all = No_of_Tweets / (all_degree_source + all_degree_dest)
    cij = No_of_Tweets / (vi + vj)

    df.at[index_df, 'Cij (All Degrees)'] = cij_all
    df.at[index_df, 'Cij'] = cij
    

In [24]:
df

,Source,Destination,Number of Tweets,All Degree of Source,All Degree of Destination,Vi - Out Degree of Source,Vj - In Degree of Destination,Percentage of Incoming at Destination (Total Tweet / Vj),Percentage of Outgoing From Source (Total Tweet / Vi),Tweets Per Population,Cij (All Degrees),Cij
0,CH,CH,111.0,387.0,387.0,208.0,179.0,62.011173,53.365385,0.000006,0.143411,0.286822
1,GB,GB,5043.0,13998.0,13998.0,7134.0,6864.0,73.47028,70.689655,0.000038,0.180133,0.360266
2,US,US,26779.0,60559.0,60559.0,30216.0,30343.0,88.254293,88.625232,0.000041,0.221098,0.442197
3,PL,PL,11.0,72.0,72.0,29.0,43.0,25.581395,37.931034,0.0,0.076389,0.152778
4,IE,IE,63.0,265.0,265.0,143.0,122.0,51.639344,44.055944,0.000006,0.118868,0.237736
...,...,...,...,...,...,...,...,...,...,...,...,...
656,MX,IE,1.0,399.0,265.0,217.0,122.0,0.819672,0.460829,0.0,0.001506,0.00295
657,SI,US,1.0,7.0,60559.0,4.0,30343.0,0.003296,25.0,0.0,0.000017,0.000033
658,ES,PT,1.0,109.0,122.0,48.0,95.0,1.052632,2.083333,0.0,0.004329,0.006993
659,TH,GB,1.0,12.0,13998.0,3.0,6864.0,0.014569,33.333333,0.0,0.000071,0.000146


In [25]:
df.to_csv('/home/dipp/Github/Master-Thesis-dipp/Misinformation_Dataset_Analysis/Network_Analysis/Result/global_network_cat_4_only_3.csv', index=False)